# Segment 3.1: Heuristic Noise Models with Cirq

**Goal:** Export Squin circuits to Cirq and apply QuEra's heuristic noise models

This demonstrates:
- Squin to Cirq conversion
- GeminiOneZone noise model
- Comparison of different noise channels
- Effect of noise model parameters

In [ ]:
from bloqade import squin
from bloqade.cirq_utils import noise
from bloqade.cirq_utils.emit import emit_circuit
from bloqade.cirq_utils import load_circuit
import bloqade.stim
import bloqade.tsim
import numpy as np

## Define Test Circuits

In [ ]:
@squin.kernel
def simple_bell_state():
    """Simple Bell state for noise analysis"""
    q = squin.qalloc(2)
    squin.h(q[0])
    squin.cx(q[0], q[1])
    squin.measure(q[0])
    squin.measure(q[1])

## Apply Heuristic Noise

In [ ]:
def apply_heuristic_noise(kernel_func, noise_scale: float = 1.0):
    """
    Apply QuEra's heuristic noise model to a circuit
    
    Args:
        kernel_func: Squin kernel function
        noise_scale: Scaling factor for all noise parameters
    
    Returns:
        Noisy Squin circuit
    """
    # Convert to Cirq
    cirq_circuit = emit_circuit(kernel_func)
    
    # Define noise model (GeminiOneZone is QuEra's default architecture)
    noise_model = noise.GeminiOneZoneNoiseModel(scaling_factor=noise_scale)
    
    # Apply noise to circuit
    noisy_cirq = noise.transform_circuit(cirq_circuit, model=noise_model)
    
    # Convert back to Squin
    noisy_squin = load_circuit(noisy_cirq)
    
    return noisy_squin

## Compare Noise Scaling Factors

In [ ]:
scaling_factors = [0.5, 1.0, 2.0, 5.0]
shots = 5000

results = []

for scale in scaling_factors:
    print(f"\n--- Noise scaling factor: {scale} ---")
    
    # Apply heuristic noise
    noisy_circuit = apply_heuristic_noise(simple_bell_state, scale)
    
    # Simulate with Stim
    stim_circ = bloqade.stim.Circuit(noisy_circuit)
    sampler = stim_circ.compile_sampler()
    samples = sampler.sample(shots=shots)
    
    # Analyze Bell state fidelity
    outcomes = {}
    for sample in samples:
        outcome = ''.join(map(str, sample.astype(int)))
        outcomes[outcome] = outcomes.get(outcome, 0) + 1
    
    # Bell state should have |00> and |11>
    ideal_count = outcomes.get('00', 0) + outcomes.get('11', 0)
    fidelity = ideal_count / shots
    
    print(f"Outcomes:")
    for outcome, count in sorted(outcomes.items(), key=lambda x: x[1], reverse=True)[:4]:
        print(f"  |{outcome}>: {count:4d} ({count/shots:.3f})")
    
    print(f"Bell state fidelity: {fidelity:.3f}")
    results.append((scale, fidelity))

# Summary
print("\n" + "="*60)
print("Summary: Fidelity vs Noise Scaling")
print("="*60)
for scale, fidelity in results:
    print(f"  Scale {scale:.1f}: Fidelity {fidelity:.3f}")

## Run Full Implementation

In [ ]:
# For complete implementation, run the Python script
import subprocess
import sys

result = subprocess.run(
    [sys.executable, '01_heuristic_noise.py'],
    capture_output=True,
    text=True
)

print(result.stdout)

## Summary

**Key Achievements:**
- ✓ Squin → Cirq → Squin conversion pipeline
- ✓ GeminiOneZone heuristic noise model
- ✓ Noise scaling analysis
- ✓ Comparison with manual noise insertion

**Next:** Segment 3.2 - Logical vs Physical Error Scaling